### 데이터 확보
1. compare data from db view, data from aureum
2. for given asset_info -> get pnu -> get jibunaddr -> get x,y
3. make igis_{} dataset : {}= fund, fund_aum, asset, lend, bene

In [1]:
import constant as c
import pandas as pd
import sys, os
import psycopg2
import warnings; warnings.filterwarnings("ignore")

In [2]:
# view_name_aureum_name_dict = c.view_name_aureum_name_dict

# constant 콜
view_name_en_col_kr_col_dict = c.view_name_en_col_kr_col_dict
ifundaum_aureum_cols = c.ifundaum_aureum_cols

# path 지정
iasset_view_path = "../asset/igisbiz/v_a_asset_info_202408271303.csv"
ibene_view_path = "../asset/igisbiz/v_a_bene_info_202408271304.csv"
ifundaum_view_path =  "../asset/igisbiz/v_a_fund_aum_mgt_202408271305.csv"
ifund_view_path = "../asset/igisbiz/v_a_fund_mgt_202408271305.csv"
ilend_view_path = "../asset/igisbiz/v_a_lend_info_202408271306.csv"

iasset_aureum_path = "../asset/igisbiz/투자 자산 조회_20240827.xlsx" # 투자자산조회
ibene_aureum_path = "../asset/igisbiz/수익자 정보 조회_20240827.xlsx"
ifundaum_aureum_path = "../asset/igisbiz/펀드 AUM 관리_20240827.xlsx"
ifund_aureum_path = "../asset/igisbiz/펀드 관리_20240827.xlsx"
ilend_aureum_path = "../asset/igisbiz/대주 정보 조회_20240827.xlsx"

iassetmgt_aureum_path = "../asset/igisbiz/투자 자산 관리_20240827.xlsx" # 투자자산관리 (view에는 없었음)

# view 데이터 객체 생성
iasset_view = pd.read_csv(iasset_view_path)
ibene_view = pd.read_csv(ibene_view_path)
ifundaum_view = pd.read_csv(ifundaum_view_path)
ifund_view = pd.read_csv(ifund_view_path)
ilend_view = pd.read_csv(ilend_view_path)

# view 데이터 열명 업그레이드
iasset_view = iasset_view.rename(columns=view_name_en_col_kr_col_dict['v_a_asset_info'])
ibene_view = ibene_view.rename(columns=view_name_en_col_kr_col_dict['v_a_bene_info'])
ifundaum_view = ifundaum_view.rename(columns=view_name_en_col_kr_col_dict['v_a_fund_aum_mgt']) 
ifund_view = ifund_view.rename(columns=view_name_en_col_kr_col_dict['v_a_fund_mgt'])
ilend_view = ilend_view.rename(columns=view_name_en_col_kr_col_dict['v_a_lend_info'])

# 아우름 데이터 객체 생성
iasset_aureum = pd.read_excel(iasset_aureum_path) # 투자자산조회
ibene_aureum = pd.read_excel(ibene_aureum_path)
ifundaum_aureum = pd.read_excel(ifundaum_aureum_path)
ifund_aureum = pd.read_excel(ifund_aureum_path)
ilend_aureum = pd.read_excel(ilend_aureum_path)
iassetmgt_aureum = pd.read_excel(iassetmgt_aureum_path) # 투자자산관리

# 아우름 데이터 형태 전처리
ibene_aureum = ibene_aureum.iloc[:-1,:]
ilend_aureum = ilend_aureum.iloc[:-1,:]
iassetmgt_aureum = iassetmgt_aureum.iloc[0:,1:] # 투자자산관리
iassetmgt_aureum_cols = iassetmgt_aureum.iloc[0,:]
iassetmgt_aureum.columns = iassetmgt_aureum_cols
iassetmgt_aureum = iassetmgt_aureum.iloc[1:,:]
ifund_aureum_cols = ifund_aureum.iloc[0,:]
def prefix_fs_to_value(series):
    return [f"FS_{value}" if '사무수탁사' in idx else value for idx, value in series.items()]
ifund_aureum_cols = prefix_fs_to_value(ifund_aureum_cols)
ifund_aureum.columns = ifund_aureum_cols
ifund_aureum = ifund_aureum.iloc[1:,:]
ifundaum_aureum.columns = ifundaum_aureum_cols
ifundaum_aureum = ifundaum_aureum.iloc[1:,:]

In [3]:
# asset 비교
print("ASSET(투자자산 조회)")
print(f"- shape:\nview={iasset_view.shape}\naureum={iasset_aureum.shape}")
print(f"- cols:\nview={sorted(list(iasset_view.columns))}\naureum={sorted(list(iasset_aureum.columns))}")
print("----------------------")
# bene 비교
print("BENE(수익자 정보 조회)")
print(f"- shape:\nview={ibene_view.shape}\naureum={ibene_aureum.shape}")
print(f"- cols:\nview={sorted(list(ibene_view.columns))}\naureum={sorted(list(ibene_aureum.columns))}")
print("----------------------")

# fundaum 비교
print("FUNDAUM(펀드 AUM 조회)")
print(f"- shape:\nview={ifundaum_view.shape}\naureum={ifundaum_aureum.shape}")
print(f"- cols:\nview={sorted(list(ifundaum_view.columns))}\naureum={sorted(list(ifundaum_aureum.columns))}")
print("----------------------")

# fund 비교 
print("FUND(펀드 조회)")
print(f"- shape:\nview={ifund_view.shape}\naureum={ifund_aureum.shape}")
print(f"- cols:\nview={sorted(list(ifund_view.columns))}\naureum={sorted(list(ifund_aureum.columns))}")
print("----------------------")

# lend 비교
print("LEND(수익자 조회)")
print(f"- shape:\nview={ilend_view.shape}\naureum={ilend_aureum.shape}")
print(f"- cols:\nview={sorted(list(ilend_view.columns))}\naureum={sorted(list(ilend_aureum.columns))}")
print("----------------------")

ASSET(투자자산 조회)
- shape:
view=(876, 42)
aureum=(476, 38)
- cols:
view=['CAP.RATE(%)', 'M²', 'NOI', '객실수(실)', '공모사모구분', '공모사모구분코드', '공실률(%)', '국내해외구분명', '국내해외구분코드', '당사펀드재간접포함여부', '대출금액', '멀티클래스구분', '멀티클래스구분코드', '모자구분', '모자구분코드', '부동산수(동)', '비고', '순번', '약칭', '여부', '완료일', '운용개시일', '운용상태', '운용상태코드', '자산명', '자산유형', '자산유형코드', '자산코드', '준공(예정일)', '투자유형', '투자유형코드', '투자자산유형', '투자전략', '투자전략코드', '투자지역', '투자지역코드', '펀드구분', '펀드구분코드', '펀드명', '펀드설정일', '펀드코드', '평']
aureum=['(개별)자산유형', '건축연한(년)', '국내/해외(자산)', '국내해외구분', '권역', '당사펀드재간접포함여부', '리모델링완료일', '만기일', '매각여부', '멀티클래스구분', '모자구분', '모집형태', '설정일', '설정환매방식', '순번', '약칭', '연면적(m²)', '연면적(평)', '운용상태', '자산(건물)명', '자산-투자유형', '재간접투자섹터', '재간접투자유형', '재간접투자전략', '전체주소(시/도, 구/군 포함)', '준공(예정)일', '투자국가', '투자섹터', '투자전략', '투자지역', '펀드구분', '펀드명', '펀드유형', '펀드코드', '펀드형태', '해지일', '현지PM', '현지운용사']
----------------------
BENE(수익자 정보 조회)
- shape:
view=(1216, 20)
aureum=(974, 22)
- cols:
view=['기준일', '만기예정일', '비고', '수익자', '수익자구분', '수익자분류', '수익자코드', '약정콜소진금액_원화', '약정콜일자', '약칭', 

In [19]:
# 운영서버에서 최신 누적 market_market, market_tenant 테이블 뷰 확보 및 저장
# Aws connect settings

# endpoint ="etlrds.cjezp4ewipgs.ap-northeast-2.rds.amazonaws.com"
# port = 5432
# database_name = "postgres"
# username =  "postgres"
# password = "dlwltm12#"

# # Establish a connection
# try:
#     connection = psycopg2.connect(
#         host=endpoint,
#         port=port,
#         dbname=database_name,
#         user=username,
#         password=password
#     )
#     print("Connected to the PostgreSQL database on AWS RDS")
#     cursor = connection.cursor()

# except psycopg2.Error as error:
#     print("Error connecting to the PostgreSQL database on AWS RDS:", error)

In [5]:
market_market_query = """
SELECT 
    m.*, 
    a.pnu, 
    a.name AS bldnm, 
    ac.lat AS y, 
    ac.lng AS x
FROM 
    market.market m
LEFT JOIN 
    market.asset a ON m.id = a.id
LEFT JOIN 
    market.asset_coord ac ON m.id = ac.asset_id;
"""


market_tenant_query = """
SELECT 
    t.*, 
    a.pnu, 
    a.name AS bldnm, 
    ac.lat AS y, 
    ac.lng AS x
FROM 
    market.tenant t
LEFT JOIN 
    market.asset a ON t.id = a.id
LEFT JOIN 
    market.asset_coord ac ON t.id = ac.asset_id;
"""

market_market = pd.read_sql(market_market_query, connection)
market_tenant = pd.read_sql(market_tenant_query, connection)

### 데이터 서머리(열별 결측률 탐색, 최빈값 등)

In [6]:
def data_summary(df: pd.DataFrame, df_name: str) -> pd.DataFrame:
    def is_missing(x):
        return x is None or x == 0.0 or x == 'NoData' or pd.isna(x) or x == ''
    
    # Initialize a list to store the result rows
    rows = []
    
    # Iterate over each column in the DataFrame
    for col in df.columns:
        # Count missing values based on the custom condition
        missing_count = df[col].apply(is_missing).sum()
        # Calculate the percentage of missing values
        missing_ratio = round(missing_count / len(df)*100, 2)
        
        # Get top 3 most frequent values
        top3_values = df[col].value_counts().head(3).index.tolist()
        top3_values_str = ','.join(map(str, top3_values))
        
        # Append the result as a new row
        rows.append([df_name, col, missing_count, missing_ratio, top3_values_str])
    
    # Convert the list to a DataFrame with specified column names
    summary_df = pd.DataFrame(rows, columns=['테이블이름', '열이름', '결측수', '결측률', '최빈값탑3'])
    
    return summary_df

#### ASSET(투자자산조회)
- 뷰테이블
1. 펀드의 정보 관련된 데이터 활용가능: 펀드코드, 약칭, 펀드명 , 운용상태
2. 존재 이유 의심하게 하는 결측률 100%열들 존재: 자산유형, 투자유형
3. M2, 평, CAP RATE 등 자산에 대한 좋은 정보를 가진 열들이 많아보이나 결측률이 높아 통계적 보완(평균치 산입 등)으로도 정제가 힘듦
4. 다만, 제일 중요한 '지번주소' , 'PNU'가 없음(종 전 뷰테이블에서는 주소지가 기입되어 있었던 것으로 기억함)

- 아우름 테이블
1. 펀드의 정보 관련된 열들 사용 가능

- 결론
1. 뷰테이블 전체 행수가 많으며, 결측률도 준수함 -> iasset_view 중심으로 연구 수행
2. pk: 자산코드, 이 외 활용열: 펀드코드, 투자자산유형, 국내해외구분코드, 투자지역, 펀드설정일 투자전략

In [7]:
# asset 비교
print("iasset_view")
print(f"shape: {iasset_view.shape}")
data_summary(iasset_view, "iasset_view")

iasset_view
shape: (876, 42)


,테이블이름,열이름,결측수,결측률,최빈값탑3
0,iasset_view,펀드코드,0,0.00,"112584,112466,112504"
1,iasset_view,약칭,0,0.00,"435호,377호,392호"
2,iasset_view,펀드명,0,0.00,"이지스일반사모투자신탁435호(태양광),이지스일반사모투자신탁377호,이지스일반사모투자..."
3,iasset_view,순번,0,0.00,"1,2,3"
4,iasset_view,자산유형코드,0,0.00,"1,15,13"
5,iasset_view,투자자산유형,2,0.23,"오피스,특별자산,복합(오피스)"
6,iasset_view,자산코드,0,0.00,"A112425001,A112621001,A112721001"
7,iasset_view,자산유형,876,100.00,
8,iasset_view,자산명,0,0.00,"Global REITs(공모),인천석남물류센터,Hines US Property 재간접"
9,iasset_view,국내해외구분코드,0,0.00,"320001,320002,320003"


In [8]:
print("iasset_aureum")
print(f"shape: {iasset_aureum.shape}")
data_summary(iasset_aureum, "iasset_aureum")

iasset_aureum
shape: (476, 38)


,테이블이름,열이름,결측수,결측률,최빈값탑3
0,iasset_aureum,펀드코드,0,0.00,"112584,112466,112504"
1,iasset_aureum,약칭,0,0.00,"435호,377호,392호"
2,iasset_aureum,펀드명,0,0.00,"이지스일반사모투자신탁435호(태양광),이지스일반사모투자신탁377호,이지스일반사모투자..."
3,iasset_aureum,국내해외구분,0,0.00,"국내,해외,혼합"
4,iasset_aureum,펀드유형,48,10.08,"대출형,실물형,역외재간접"
5,iasset_aureum,투자섹터,0,0.00,"특별자산,오피스,주거"
6,iasset_aureum,순번,0,0.00,"1,2,3"
7,iasset_aureum,자산(건물)명,0,0.00,"Hines US Property 재간접,Brookfield Super-Core In..."
8,iasset_aureum,자산-투자유형,0,0.00,"대출,실물,재간접"
9,iasset_aureum,(개별)자산유형,0,0.00,"특별자산,역외재간접,오피스"


#### BENE(수익자 정보 조회)
- 뷰테이블
1. 전반적으로 결측률이 낮아 활용 가능성 있음
2. 다만, 수익자구분 열의 결측률이 높아서 아쉬운 점

- 아우름 테이블
1. 결측률이 70%에 육박하긴 하나 뷰에는 없는 '설정해지'에 관한 열들이 있음

- 결론
1. 뷰테이블 활용
2. pk: 펀드코드 주요열: 수익자, 이 외 활용열: 펀드코드, 투자자산유형, 펀드설정일, 수익자, 수익자구분, 수익자분류, 최초약정일, 총약정금액


- To Do
1. 뷰테이블에 수익자코드 A5F067, A5F056 등 데이터가 있는 것으로 확인되는데 우선 뷰, 아우름에는 정보가 없는 것으로 확인됨 -> DSU에 엔터티 관련 테이블이 따로 관리되고 있는지 문의(메일 전송완료 24/08/27)

In [9]:
# bene 비교
print("ibene_view")
print(f"shape: {ibene_view.shape}")
data_summary(ibene_view, "ibene_view")

ibene_view
shape: (1216, 20)


,테이블이름,열이름,결측수,결측률,최빈값탑3
0,ibene_view,기준일,0,0.00,"2024-07-31,2022-04-30,2024-03-31"
1,ibene_view,펀드코드,0,0.00,"112475,112614,112321"
2,ibene_view,약칭,0,0.00,"372호,421호,297호"
3,ibene_view,펀드명,0,0.00,"이지스제372호전문투자형사모부동산투자신탁(운용),이지스일반사모부동산투자신탁421호(..."
4,ibene_view,투자자산유형코드,3,0.25,"310001.0,310005.0,310006.0"
5,ibene_view,투자자산유형,3,0.25,"오피스,주거,복합(오피스)"
6,ibene_view,자산명,59,4.85,"부실채권을 기초자산으로 하는 SPC의 출자지분 및 사채,Hines US Proper..."
7,ibene_view,운용상태코드,0,0.00,"100002,100004"
8,ibene_view,운용상태,0,0.00,"운용,청산"
9,ibene_view,펀드설정일,0,0.00,"2021-11-24,2016-11-28,2020-08-28"


In [10]:
print("ibene_aureum")
print(f"shape: {ibene_aureum.shape}")
data_summary(ibene_aureum, "ibene_aureum")

ibene_aureum
shape: (974, 22)


,테이블이름,열이름,결측수,결측률,최빈값탑3
0,ibene_aureum,기준일자,0,0.00,2024-07-31
1,ibene_aureum,펀드코드,0,0.00,"112475,112614,112451"
2,ibene_aureum,약칭,0,0.00,"372호,421호,362호"
3,ibene_aureum,펀드명,0,0.00,"이지스제372호전문투자형사모부동산투자신탁(운용),이지스일반사모부동산투자신탁421호(..."
4,ibene_aureum,투자섹터,0,0.00,"오피스,주거,복합(오피스)"
5,ibene_aureum,자산,40,4.11,"Hines US Property 재간접,CJ제일제당센터,와이디427피에프브이 주식회사"
6,ibene_aureum,운용상태,0,0.00,"운용,청산"
7,ibene_aureum,국내해외구분,0,0.00,"국내,해외,혼합"
8,ibene_aureum,설정일,0,0.00,"2021-11-24,2021-11-22,2020-08-28"
9,ibene_aureum,만기일,11,1.13,"2031-11-22,2025-08-28,2026-08-31"


#### FUNDAUM(펀드 AUM)
- 뷰테이블
1. 뷰테이블은 기준일자별 펀드의 AUM을 담고 있음 -> 행수가 많음

- 아우름 테이블
1. 아우름테이블은 특정일자의 펀드의 AUM을 담고 있음
2. 열이 훨씬 많음(이는 아우름 테이블에 열 추가를 요청하는 다른 부서들이 많아서로 확인됨)

- 결론
1. 뷰테이블 활용하되 특정 기준일자를 기준으로 데이터를 필터링해서 분석을 수행 
2. pk: 펀드코드 이 외 활용열: 약칭, 자산명, 자산갯수, 투자자산유형, EQUITY총액_약정, LOAN총액_약정, 기준일자보증금_약정

In [11]:
# fundaum 비교
print("ifundaum_view")
print(f"shape: {ifundaum_view.shape}")
data_summary(ifundaum_view, "ifundaum_view")

ifundaum_view
shape: (15543, 31)


,테이블이름,열이름,결측수,결측률,최빈값탑3
0,ifundaum_view,기준일자,0,0.00,"2024-07-31,2024-06-30,2024-05-31"
1,ifundaum_view,펀드코드,0,0.00,"112005,112451,112482"
2,ifundaum_view,약칭,0,0.00,"6호,362호,366호"
3,ifundaum_view,펀드명,0,0.00,"이지스전문투자형사모부동산투자신탁6호,이지스전문투자형사모부동산투자신탁362호,이지스글..."
4,ifundaum_view,자산명,616,3.96,"Global REITs(공모),Hines US Property 재간접,Poland ..."
5,ifundaum_view,자산갯수,616,3.96,"1.0,3.0,2.0"
6,ifundaum_view,자산면적합계(평),6690,43.04,"36434.31,72354.2793,24089.26"
7,ifundaum_view,투자자산유형코드,24,0.15,"310001.0,310006.0,310005.0"
8,ifundaum_view,투자자산유형,24,0.15,"오피스,복합(오피스),주거"
9,ifundaum_view,운용상태코드,0,0.00,"100002,100004"


In [12]:
print("ifundaum_aureum")
print(f"shape: {ifundaum_aureum.shape}")
data_summary(ifundaum_aureum, "ifundaum_aureum")

ifundaum_aureum
shape: (352, 57)


,테이블이름,열이름,결측수,결측률,최빈값탑3
0,ifundaum_aureum,펀드코드,0,0.00,"112005,112006,112686"
1,ifundaum_aureum,약칭,0,0.00,"6호,7호,469-2호"
2,ifundaum_aureum,펀드명,1,0.28,"이지스전문투자형사모부동산투자신탁6호,연기금글로벌리츠일반사모부동산투자신탁1호(리츠-재..."
3,ifundaum_aureum,투자자산유형,1,0.28,"오피스,복합(오피스),주거"
4,ifundaum_aureum,자산수,24,6.82,"1,3,2"
5,ifundaum_aureum,연면적(평),179,50.85,"90541,40918.59,24416.89"
6,ifundaum_aureum,자산명,24,6.82,"Hines US Property 재간접,Brookfield Super-Core In..."
7,ifundaum_aureum,운용상태,1,0.28,"운용,청산"
8,ifundaum_aureum,설정일,1,0.28,"2021-12-27,2021-11-24,2022-07-18"
9,ifundaum_aureum,만기일,7,1.99,"9999-12-31,2099-12-31,2032-07-18"


#### FUND(펀드)
- 뷰테이블
1. 펀드 관련 정보의 기준이 되는 테이블인 것으로 확인

- 아우름 테이블
1. 스카이펀드에서 물려온 데이터 존재

- 결론
1. 펀드 테이블은 brbasis와 같이 펀드와 관련된 전반적인 내용을 담은 기준 테이블인 것으로 확인됨. 데이터 분석의 경우 어차피 주제가 깊게(AUM, 대주, 수익자, ..) 등으로 선정될 것이기에 본 테이블은 활용하기가 애매한 것으로 판단함
2. 뷰테이블 활용
3. pk: 펀드코드 주요활용열: 펀드명, 자산명, 투자자산유형, 펀드구분, 펀드설정일, 투자유형, 투자전략 등

In [13]:
# fund 비교 
print("ifund_view")
print(f"shape: {ifund_view.shape}")
data_summary(ifund_view, "ifund_view")

ifund_view
shape: (945, 54)


,테이블이름,열이름,결측수,결측률,최빈값탑3
0,ifund_view,펀드코드,0,0.00,"112001,112649,112634"
1,ifund_view,약칭,3,0.32,"R레지던스,1호,453호"
2,ifund_view,펀드명,0,0.00,"이지스전문투자형사모부동산투자신탁1호,이지스부산특구부동산투자신탁제1호(운용),이지스리..."
3,ifund_view,자산명,262,27.72,"Global REITs(공모),Hines US Property 재간접,인천석남물류센터"
4,ifund_view,수익자 정보,552,58.41,"대한지방행정공제회,한국교직원공제회,농업협동조합중앙회"
5,ifund_view,대주정보,798,84.44,"삼성생명,미정,삼성생명, 신한은행"
6,ifund_view,운용상태코드,0,0.00,"100002,100004,100005"
7,ifund_view,운용상태,0,0.00,"운용,청산,미설정"
8,ifund_view,개발여부,0,0.00,"N,Y"
9,ifund_view,국내해외구분코드,67,7.09,"320001.0,320002.0,320003.0"


In [14]:
print("ifund_aureum")
print(f"shape: {ifund_aureum.shape}")
data_summary(ifund_aureum, "ifund_aureum")

ifund_aureum
shape: (945, 64)


,테이블이름,열이름,결측수,결측률,최빈값탑3
0,ifund_aureum,펀드코드,0,0.00,"112001,112649,112634"
1,ifund_aureum,약칭,3,0.32,"R레지던스,1호,453호"
2,ifund_aureum,펀드명,0,0.00,"이지스전문투자형사모부동산투자신탁1호,이지스부산특구부동산투자신탁제1호(운용),이지스리..."
3,ifund_aureum,펀드영문명,379,40.11,IGIS Platform KDCIP 1 Real Estate Professional...
4,ifund_aureum,자산명,262,27.72,"Global REITs(공모),Hines US Property 재간접,인천석남물류센터"
...,...,...,...,...,...
59,ifund_aureum,FS_집합투자기구분류,134,14.18,"17151Z42A721049ZZZZB,17141Y42A720142ZZZZ2,1714..."
60,ifund_aureum,FS_KSD펀드코드,138,14.60,"035470112722,035470112425,035470112629"
61,ifund_aureum,FS_KSD종목코드,244,25.82,"KRZ501829161,KRZ500669880,KRZ502479022"
62,ifund_aureum,FS_결산기준일,934,98.84,"2024-12-31,2024-03-31,2023-06-27"


#### lend(대주정보조회)
- 뷰테이블
1. 기준금리, 스프레드 제외하고는 전반적으로 결측률 0에 수렴

- 아우름 테이블
1. 뷰테이블과 열구성, 결측률 등 거의 유사

- 결론
1. 뷰테이블 활용
2. pk: 펀드코드(가능하다면 대주코드까지 엮어서) 주요활용열: 대주코드, 대주명, 대출약정금액, 대출실행일, 차입유형, 이자유형, 대출유형 펀드코드

In [15]:
# lend 비교
print("ilend_view")
print(f"shape: {ilend_view.shape}")
data_summary(ilend_view, "ilend_view")

ilend_view
shape: (875, 29)


,테이블이름,열이름,결측수,결측률,최빈값탑3
0,ilend_view,기준일,0,0.00,"2024-07-31,2022-10-31,2022-06-30"
1,ilend_view,펀드코드,0,0.00,"P00018,P00030,112579"
2,ilend_view,약칭,0,0.00,"P마곡CP4,P와이디427,420호"
3,ilend_view,펀드명,0,0.00,"마곡CP4피에프브이 주식회사,와이디427피에프브이 주식회사,이지스제420호전문투자형..."
4,ilend_view,투자자산유형코드,0,0.00,"310006,310001,310005"
5,ilend_view,투자자산유형,0,0.00,"복합(오피스),오피스,주거"
6,ilend_view,자산명,0,0.00,"마곡CP4 복합시설 개발,밀레니엄힐튼호텔,마곡 명소화부지 개발사업"
7,ilend_view,운용상태코드,0,0.00,"100002,100004"
8,ilend_view,운용상태,0,0.00,"운용,청산"
9,ilend_view,펀드설정일,0,0.00,"2021-11-24,2020-03-12,2021-08-11"


In [16]:
print("ilend_aureum")
print(f"shape: {ilend_aureum.shape}")
data_summary(ilend_aureum, "ilend_aureum")

ilend_aureum
shape: (820, 24)


,테이블이름,열이름,결측수,결측률,최빈값탑3
0,ilend_aureum,기준일자,0,0.00,2024-07-31
1,ilend_aureum,펀드코드,0,0.00,"P00018,P00030,112579"
2,ilend_aureum,약칭,0,0.00,"P마곡CP4,P와이디427,420호"
3,ilend_aureum,펀드명,0,0.00,"마곡CP4피에프브이 주식회사,와이디427피에프브이 주식회사,이지스제420호전문투자형..."
4,ilend_aureum,투자섹터,0,0.00,"복합(오피스),오피스,주거"
5,ilend_aureum,자산,0,0.00,"마곡CP4 복합시설 개발,밀레니엄힐튼호텔,마곡 명소화부지 개발사업"
6,ilend_aureum,운용상태,0,0.00,"운용,청산"
7,ilend_aureum,국내해외구분,0,0.00,"국내,해외"
8,ilend_aureum,설정일,0,0.00,"2021-11-24,2020-03-12,2021-08-11"
9,ilend_aureum,만기일,25,3.05,"2040-03-12,2031-11-23,2037-07-29"


### 데이터 보완(지번주소 추가/수정, PNU 추가, x,y 좌표 추가)
- 영욱님 제공해주신 자료 활용(아우름_에너지사용량_230829_월별데이터.xlsx): 여기에 이지스 펀드코드 - 지번주소 - PNU 맵핑 내역이 있음
- iasset.csv 로 만들어서 BDDB.address 테이블 정보에 like문으로 쿼리하여 풀 지번주소 추출 수행

In [17]:
path = "../asset/igisbiz/fundcd_jibunaddr_pnu_240827.xlsx"
fundcd_jibundaddr_pnu = pd.read_excel(path, dtype=str)
fundcd_jibundaddr_pnu['assetcd'] = 'A' + fundcd_jibundaddr_pnu['k'].str[:6] + fundcd_jibundaddr_pnu['k'].str[-1].str.zfill(3)

path = "../asset/igisbiz/iasset_pnu_added.csv"
iasset = pd.read_csv(path)
iasset_view = iasset_view.merge(iasset[['assetcd', 'platplc', 'pnu']], how='left', left_on='자산코드', right_on='assetcd').drop('assetcd', axis=1)

### 보완 데이터테이블들 저장

In [18]:
iasset_view.to_csv("../asset/igisbiz/iasset_view_202408.csv", index=False)
ibene_view.to_csv("../asset/igisbiz/ibene_view_202408.csv", index=False)
ifundaum_view.to_csv("../asset/igisbiz/ifundaum_view_202408.csv", index=False)
ifund_view.to_csv("../asset/igisbiz/ifund_view_202408.csv", index=False)
ilend_view.to_csv("../asset/igisbiz/ilend_view_202408.csv", index=False)
iasset_aureum.to_csv("../asset/igisbiz/iasset_aureum_202408.csv", index=False)
ibene_aureum.to_csv("../asset/igisbiz/ibene_aureum_202408.csv", index=False)
ifundaum_aureum.to_csv("../asset/igisbiz/ifundaum_aureum_202408.csv", index=False)
ifund_aureum.to_csv("../asset/igisbiz/ifund_aureum_202408.csv", index=False)
ilend_aureum.to_csv("../asset/igisbiz/ilend_aureum_202408.csv", index=False)
iassetmgt_aureum.to_csv("../asset/igisbiz/iassetmgt_aureum_202408.csv", index=False)
market_market.to_csv("../asset/igisbiz/market_market_202408.csv", index=False)
market_tenant.to_csv("../asset/igisbiz/market_tenant_202408.csv", index=False)